In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt

# Fungsi untuk memformat nominal uang
def format_currency(x, pos):
    if x >= 1e9:
        return f'{x/1e9:.1f}B'
    elif x >= 1e6:
        return f'{x/1e6:.1f}M'
    elif x >= 1e3:
        return f'{x/1e3:.1f}K'
    else:
        return f'{x:.0f}'
    
# Mengambil data dari API
response = requests.get("http://localhost:3000/api/sales")
data = pd.read_json(response.text)

# Mengubah kolom 'date' menjadi tipe datetime
data['date'] = pd.to_datetime(data['date'])

# Menghitung pendapatan
data['revenue'] = data['price'] * data['quantity']

# Loop untuk menampilkan data grafik dari bulan Januari sampai Desember
for bulan in range(1, 13):
    # Filter data untuk bulan tertentu
    tanggal_awal = f'2024-{bulan:02d}-01'
    tanggal_akhir = f'2024-{bulan:02d}-{pd.Timestamp(tanggal_awal).days_in_month}'
    data_filter = data[(data['date'] >= tanggal_awal) & (data['date'] <= tanggal_akhir)]

    # Analisis pendapatan harian
    data_daily = data_filter.groupby(data_filter['date'].dt.date)['revenue'].sum().reset_index()
    data_daily.columns = ['date', 'daily_revenue']

    # Membuat grafik pendapatan harian
    plt.figure(figsize=(12, 6))
    bars = plt.bar(data_daily['date'], data_daily['daily_revenue'])
    plt.title(f'Pendapatan Harian - {pd.Timestamp(tanggal_awal).strftime("%B %Y")}')
    plt.xlabel('Tanggal')
    plt.ylabel('Pendapatan (Rp)')
    plt.xticks(rotation=45)
    plt.grid(True, linestyle='--', alpha=0.7)

    # Menambahkan nominal pada atas chart
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                 f'{format_currency(height, 0)}',
                 ha='center', va='bottom', rotation=0)

    plt.tight_layout()
    plt.show()

    # Analisis pendapatan tertinggi dan terendah
    max_daily_revenue = data_daily['daily_revenue'].max()
    min_daily_revenue = data_daily['daily_revenue'].min()
    max_daily_date = data_daily.loc[data_daily['daily_revenue'].idxmax(), 'date']
    min_daily_date = data_daily.loc[data_daily['daily_revenue'].idxmin(), 'date']

    # Menampilkan hasil analisis
    print(f"\nAnalisis Pendapatan Harian - {pd.Timestamp(tanggal_awal).strftime('%B %Y')}:")
    print(f"Total pendapatan: Rp{data_daily['daily_revenue'].sum():,.0f}")
    print(f"Rata-rata pendapatan harian: Rp{data_daily['daily_revenue'].mean():,.0f}")
    print(f"Pendapatan harian tertinggi: Rp{max_daily_revenue:,.0f} pada {max_daily_date}")
    print(f"Pendapatan harian terendah: Rp{min_daily_revenue:,.0f} pada {min_daily_date}")


## Start Analisis Bulanan
# Analisis pendapatan bulanan
data_monthly = data.groupby(data['date'].dt.to_period('M'))['revenue'].sum().reset_index()
data_monthly.columns = ['month', 'monthly_revenue']
data_monthly['month'] = data_monthly['month'].dt.to_timestamp()

# Membuat grafik pendapatan bulanan
plt.figure(figsize=(12, 6))
bars = plt.bar(data_monthly['month'], data_monthly['monthly_revenue'])
plt.title('Pendapatan Bulanan')
plt.xlabel('Bulan')
plt.ylabel('Pendapatan (Rp)')
plt.xticks(data_monthly['month'], [date.strftime('%B %Y') for date in data_monthly['month']], rotation=45)
plt.grid(True, linestyle='--', alpha=0.7, axis='y')

# Menambahkan nominal pada atas chart
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{format_currency(height, 0)}',
             ha='center', va='bottom', rotation=0)

plt.tight_layout()
plt.show()

# Analisis pendapatan tertinggi dan terendah
max_monthly_revenue = data_monthly['monthly_revenue'].max()
min_monthly_revenue = data_monthly['monthly_revenue'].min()
max_monthly_date = data_monthly.loc[data_monthly['monthly_revenue'].idxmax(), 'month']
min_monthly_date = data_monthly.loc[data_monthly['monthly_revenue'].idxmin(), 'month']

# Menampilkan hasil analisis
print("\nAnalisis Pendapatan Bulanan:")
print(f"Total pendapatan: Rp{data_monthly['monthly_revenue'].sum():,.0f}")
print(f"Rata-rata pendapatan bulanan: Rp{data_monthly['monthly_revenue'].mean():,.0f}")
print(f"Pendapatan bulanan tertinggi: Rp{max_monthly_revenue:,.0f} pada {max_monthly_date.strftime('%B %Y')}")
print(f"Pendapatan bulanan terendah: Rp{min_monthly_revenue:,.0f} pada {min_monthly_date.strftime('%B %Y')}")